In [39]:
import numpy as np
import pickle

def multi_key_dict_get(d, k):
    for keys, v in d.items():
        if k in keys:
            return v
    return None

#64卦、4096種卦爻組合資料庫，爻由底(左)至上(右)起
sixtyfour_gua_list = pickle.load( open( "data/sixtyfour_gua_list.pkl", "rb" ) )
bagua =  pickle.load( open( "data/bagua.pkl", "rb" ) )
def bookgua(): #由底至上起爻
    shifa_results = []
    for i in range(6):
        n=2
        #掛一
        stalks_first = 50-1
        #一變 (分二、掛一、揲四、歸奇)
        first_division = np.random.multinomial(stalks_first, np.ones(n)/2, size=1)[0]
        guayi = 1
        right = first_division[0] - guayi
        left_extract = first_division[1] % 4 
        if left_extract == 0:
            left_extract = 4
        right_extract = right % 4
        if right_extract == 0:
            right_extract = 4
        yibian  = left_extract + right_extract + guayi
        #二變 (分二、掛一、揲四、歸奇)
        stalks_second = stalks_first - yibian
        second_division = np.random.multinomial(stalks_second, np.ones(n)/2, size=1)[0]
        right_second = second_division[0] - guayi
        left_extract_second = second_division[1] % 4 
        if left_extract_second == 0:
            left_extract_second = 4
        right_extract_second = right_second % 4 
        if right_extract_second == 0:
            right_extract_second = 4
        erbian = left_extract_second + right_extract_second + guayi
        #三變 (分二、掛一、揲四、歸奇)
        stalks_third = stalks_second - erbian
        third_division = np.random.multinomial(stalks_third, np.ones(n)/2, size=1)[0]
        right_third = third_division[0] - guayi
        left_extract_third = third_division[1] % 4
        if left_extract_third  == 0:
            left_extract_third = 4
        right_extract_third = right_third % 4 
        if right_extract_third == 0:
            right_extract_third = 4
        sanbian = left_extract_third + right_extract_third + guayi
        yao = int((stalks_first - yibian - erbian - sanbian) / 4)
        shifa_results.append(yao)
    return str(shifa_results[0]) + str(shifa_results[1]) + str(shifa_results[2]) + str(shifa_results[3]) + str(shifa_results[4]) + str(shifa_results[5])

def bookgua_details():
    guayao = bookgua()
    getgua = multi_key_dict_get(sixtyfour_gua_list, guayao)
    yao_results = bagua.get(getgua)
    bian_yao = guayao.replace("6","1").replace("9","1").replace("7","0").replace("8","0")
    dong_yao = bian_yao.count("1")
    explain = "動爻有【"+str(dong_yao )+"】根。"
    dong_yao_change = guayao.replace("6","7").replace("9","8")
    dong_yao_bian_gua = multi_key_dict_get(sixtyfour_gua_list, dong_yao_change)
    g_gua_result = bagua.get(dong_yao_bian_gua)
    g_gua = "【"+getgua+"之"+dong_yao_bian_gua+"】"
    top_bian_yao = bian_yao.rfind("1")+int(1)
    second_bian_yao = bian_yao.rfind("1",0, bian_yao.rfind("1"))+int(1)
    top_jing_yao = bian_yao.rfind("0") + int(1)
    second_jing_yao = bian_yao.rfind("0", 0, bian_yao.rfind("0"))+int(1)
    top = yao_results.get(top_bian_yao)
    second = yao_results.get(second_bian_yao)
    top_2 = yao_results.get(top_jing_yao)
    second_2 = yao_results.get(second_jing_yao)
    explaination2 = None
    try:
        if dong_yao == 0:
            explaination2 = "主要看【"+getgua+"】卦卦辭。",  yao_results[0]
        elif dong_yao == 1: 
            explaination2 = g_gua, "主要看【"+top[0]+top[1]+"】",  top
        elif dong_yao == 2:
            explaination2 = g_gua, "主要看【"+top[0]+top[1]+"】，其次看【"+second[0]+second[1]+"】。", top, second
        elif dong_yao == 3:
            if bian_yao.find("1") == 0:
                explaination2 = g_gua, "【"+getgua+"】卦為貞(我方)，【"+dong_yao_bian_gua+"】卦為悔(他方)。主悔【"+dong_yao_bian_gua+"】卦", g_gua_result[0],  yao_results[0]
            elif bian_yao.find("1") > 0:
                explaination2 = g_gua, "【"+getgua+"】卦為貞(我方)，【"+dong_yao_bian_gua+"】卦為悔(他方)。主貞【"+getgua+"】卦", yao_results[0], g_gua_result[0]
        elif dong_yao == 4:
            explaination2 = g_gua, "主要看【"+getgua+"】的"+top_2[0]+top_2[1]+"，其次看【"+second_2[0]+second_2[1]+"】。", top_2, second_2
        elif dong_yao == 5:    
            explaination2 = g_gua, "主要看【"+top_2[0]+top_2[1]+"】。",  top_2
        elif dong_yao == 6:
            explaination2 = g_gua, "主要看【"+dong_yao_bian_gua+"】卦。", g_gua_result[0]
    except (TypeError, UnboundLocalError):
        pass
    return  guayao, getgua, yao_results, explain, explaination2

In [40]:
alist=[]
for i in range(5000):
    a= bookgua_details()
    alist.append(a)
alist

[('987889',
  '賁',
  {0: '亨。 小利有所往。',
   1: '初九：賁其趾，舍車而徒。',
   2: '六二：賁其須。',
   3: '九三：賁如濡如，永貞吉。',
   4: '六四：賁如皤如，白馬翰如，匪寇婚媾。',
   5: '六五：賁于丘園，束帛戔戔，吝，終吉。',
   6: '上九：白賁，無咎。'},
  '動爻有【2】根。',
  ('【賁之謙】', '主要看【上九】，其次看【初九】。', '上九：白賁，無咎。', '初九：賁其趾，舍車而徒。')),
 ('789989',
  '離',
  {0: '利貞，亨。 畜牝牛，吉。',
   1: '初九：履錯然，敬之無咎。',
   2: '六二：黃離，元吉。',
   3: '九三：日昃之離，不鼓缶而歌，則大耋之嗟，凶。',
   4: '九四：突如其來如，焚如，死如，棄如。',
   5: '六五：出涕沱若，戚嗟若，吉。',
   6: '上九：王用出征，有嘉折首，獲其匪丑，無咎。'},
  '動爻有【3】根。',
  ('【離之復】',
   '【離】卦為貞(我方)，【復】卦為悔(他方)。主貞【離】卦',
   '利貞，亨。 畜牝牛，吉。',
   '亨。 出入無疾，朋來無咎。 反復其道，七日來復，利有攸往。')),
 ('978879',
  '中孚',
  {0: '豚魚吉，利涉大川，利貞。',
   1: '初九：虞吉，有他不燕。',
   2: '九二：鳴鶴在陰，其子和之，我有好爵，吾與爾靡之。',
   3: '六三：得敵，或鼓或罷，或泣或歌。',
   4: '六四：月幾望，馬匹亡，無咎。',
   5: '九五：有孚攣如，無咎。',
   6: '上九：翰音登於天，貞凶。'},
  '動爻有【2】根。',
  ('【中孚之坎】', '主要看【上九】，其次看【初九】。', '上九：翰音登於天，貞凶。', '初九：虞吉，有他不燕。')),
 ('798877',
  '中孚',
  {0: '豚魚吉，利涉大川，利貞。',
   1: '初九：虞吉，有他不燕。',
   2: '九二：鳴鶴在陰，其子和之，我有好爵，吾與爾靡之。',
   3: '六三：得敵，或鼓或罷，或泣或歌。',
   4: '六四：月幾望，馬匹亡，無咎。',
   5: '九五：有孚攣如

0